In [71]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.display import display as dsp

In [72]:
crime = pd.read_excel('data/table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.xls')

crime.columns = crime.iloc[3]
crime.drop(crime.index[[0,1,2,3]], inplace = True)
crime.index = range(len(crime))
crime.rename(index = str, columns = {'Violent\ncrime' : 'Violent Crimes',
              'Murder and\nnonnegligent\nmanslaughter': 'Murder',
              'Rape\n(revised\ndefinition)1': 'Rape Old', 
              'Rape\n(legacy\ndefinition)2' : 'Rape New',
              'Aggravated\nassault' : 'Aggrevated Assault', 
              'Property\ncrime' : 'Property Crime',
              'Larceny-\ntheft' : 'Larceny-Theft', 
              'Motor\nvehicle\ntheft' : 'Motor Theft', 'Arson3' : 'Arson'}, inplace = True)

dsp(crime.head())
print(crime['Robbery'].unique())
print(crime.columns)

3,City,Population,Violent Crimes,Murder,Rape Old,Rape New,Robbery,Aggrevated Assault,Property Crime,Burglary,Larceny-Theft,Motor Theft,Arson
0,Adams Village,1861,0,0,NaN,0,0,0,12,2,10,0,0
1,Addison Town and Village,2577,3,0,NaN,0,0,3,24,3,20,1,0
2,Akron Village,2846,3,0,NaN,0,0,3,16,1,15,0,0
3,Albany,97956,791,8,NaN,30,227,526,4090,705,3243,142,NaN
4,Albion Village,6388,23,0,NaN,3,4,16,223,53,165,5,NaN


[0 227 4 3 31 12 22 1 13 101 10 5 1322 9 2 48 17 11 8 29 7 76 19 6 60 16
 204 51 39 20 25 24 47 228 150 15 81 19170 166 21 100 26 918 23 203 34 400
 145 102 28 390 nan]
Index(['City', 'Population', 'Violent Crimes', 'Murder', 'Rape Old',
       'Rape New', 'Robbery', 'Aggrevated Assault', 'Property Crime',
       'Burglary', 'Larceny-Theft', 'Motor Theft', 'Arson'],
      dtype='object', name=3)


In [73]:
pop2 = crime['Population'] ** 2
pop2.rename("Pop^2", inplace = True)

features = pd.concat([crime['City'], crime['Population'], pop2, 
                      crime['Murder'], crime['Robbery']], axis = 1)

features.dropna(inplace = True)
#features.rename({2: 'Pop^2'}, inplace = True)
#features.rename(columns = {features.columns[2] : 'Pop^2'}, inplace = True)
dsp(features.head())

,City,Population,Pop^2,Murder,Robbery
0,Adams Village,1861,3463321,0,0
1,Addison Town and Village,2577,6640929,0,0
2,Akron Village,2846,8099716,0,0
3,Albany,97956,9595377936,8,227
4,Albion Village,6388,40806544,0,4


In [75]:
features['BinMurder'] = (features['Murder'].apply(lambda x : np.where(x > 0, 1, 0)))
features['BinRobbery'] = (features['Robbery'].apply(lambda x : np.where(x > 0, 1, 0)))
dsp(features.head())

<class 'pandas.core.series.Series'>
<class 'int'>


,City,Population,Pop^2,Murder,Robbery,BinMurder
0,Adams Village,1861,3463321,0,0,1
1,Addison Town and Village,2577,6640929,0,0,1
2,Akron Village,2846,8099716,0,0,1
3,Albany,97956,9595377936,8,227,0
4,Albion Village,6388,40806544,0,4,1
